In [1]:
import pytz 
import psycopg2
from psycopg2 import sql
import yfinance as yf
import pandas as pd
import numpy as np
import datetime

def cargar_datos_yfinance_a_postgresql():
    df = pd.read_csv(r"C:\Users\manue\Desktop\data_eng\data_eng\_02_yahoof\symbol_name\symbol_name.csv")
    tickers = df['symbol'].tolist()

    end_date = datetime.datetime.now()
    start_date = end_date - datetime.timedelta(days=1)

    all_data = pd.DataFrame()

    for ticker in tickers:
        try:
            data = yf.download(ticker, start=start_date, end=end_date, interval="5m")
            data['Symbol'] = ticker
            all_data = pd.concat([all_data, data])
        except Exception as e:
            print(f"No se pudieron obtener los datos para el ticker {ticker}: {e}")

    all_data['date_time'] = all_data.index
    all_data = all_data.reset_index(drop=True)
    all_data = all_data.rename(columns=lambda x: x.lower().replace(' ', '_'))
    all_data = all_data.rename(columns={'symbol': 'ticker'})
    all_data['date_time'] = pd.to_datetime(all_data['date_time']).dt.tz_localize(None)

    dbname = 'yfinance'
    user = 'postgres'
    password = 'mjose2002'
    host = 'localhost'
    port = '5432'

    try:
        conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )
        print("Conexión exitosa a la base de datos")
    except psycopg2.Error as e:
        print("Error al conectarse a la base de datos:", e)
        exit()

    cursor = conn.cursor()

    query = "SELECT MAX(date_time) FROM stock_data;"
    cursor.execute(query)
    ultima_fecha_cargada = cursor.fetchone()[0]
    print(ultima_fecha_cargada)

    nuevos_datos = all_data[all_data['date_time'] > ultima_fecha_cargada]

    for index, row in nuevos_datos.iterrows():
        try:
            cursor.execute(
                "INSERT INTO stock_data (open, high, low, close, adj_close, volume, ticker, date_time) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
                (row['open'], row['high'], row['low'], row['close'], row['adj_close'], row['volume'], row['ticker'], row['date_time'])
            )
        except psycopg2.Error as e:
            print("Error al insertar datos:", e)
            conn.rollback()
        else:
            conn.commit()
    conn.close()




[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Conexión exitosa a la base de datos
<cursor object at 0x000001B14B4E66C0; closed: 0>
2024-03-11 15:55:00
